In [ ]:
import numpy as np
import plotly.graph_objects as go

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.ops import MLP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# autoreload
%load_ext autoreload
%autoreload 2

Learn the dynamics $\frac{\partial p}{\partial t}$ of the path 

In [ ]:
dyn = MLP(in_channels=1, hidden_channels=[256, 256, 2]).to(device)

x_0 = torch.tensor([0.0, 0.0], device=device)

# Integrate the dynamics from t=0 to 1
T = torch.linspace(0, 1, 100, device=device)[:,None]
dx = dyn(T)
x = torch.cumsum(dx, dim=0) + x_0

fig = go.Figure()
fig.add_trace(go.Scatter(x=x[:, 0].detach().cpu().numpy(), y=x[:, 1].detach().cpu().numpy(), mode='markers+lines'))
fig.update_layout(width=500, height=500, scene_aspectmode='data')
fig.show()

Add a cost for reaching a target point $x_f$ at terminal time.

In [ ]:
# Train the dynamics to reach the goal
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(dyn.parameters(), lr=1e-3)

x_f = torch.tensor([1.0, 1.0], device=device)

for i in range(500):
    dx = dyn(T)
    loss = criterion(torch.sum(dx, dim=0), x_f)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print(f'Loss: {loss.item()}')

In [ ]:
with torch.no_grad():
    dx = dyn(T)
x = torch.cumsum(dx, dim=0) + x_0

fig = go.Figure()
fig.add_trace(go.Scatter(x=x[:, 0].detach().cpu().numpy(), y=x[:, 1].detach().cpu().numpy(), mode='markers+lines'))
fig.update_layout(width=500, height=500, scene_aspectmode='data')
fig.show()

Add a cost for minimizing arc length (integral of velocity)

In [ ]:
# Minimize distance

for i in range(500):
    dx = dyn(T)
    goal_loss = criterion(torch.sum(dx, dim=0), x_f)
    dist_loss = torch.norm(dx, dim=1).nanmean()
    loss = goal_loss + dist_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print(f'Loss: {loss.item()}')

In [ ]:
with torch.no_grad():
    dx = dyn(T)
x = torch.cumsum(dx, dim=0) + x_0

fig = go.Figure()
fig.add_trace(go.Scatter(x=x[:, 0].detach().cpu().numpy(), y=x[:, 1].detach().cpu().numpy(), mode='markers+lines'))
fig.update_layout(width=500, height=500, scene_aspectmode='data')
fig.show()

Now add another loss based on a scalar cost field

In [ ]:
# Train the dynamics to minimize cost over a scalar field

def cost(x):
    return F.relu(1.0 - torch.norm(x - torch.tensor([0.5, 0.5], device=device), dim=1))

In [ ]:
# Plot cost samples
samples = torch.rand(1000, 2, device=device)
costs = cost(samples)

fig = go.Figure()
fig.add_trace(go.Scatter(x=samples[:, 0].detach().cpu().numpy(), y=samples[:, 1].detach().cpu().numpy(), mode='markers', marker=dict(color=costs.detach().cpu().numpy(), colorscale='Viridis')))
fig.update_layout(width=500, height=500, scene_aspectmode='data')
fig.show()

In [ ]:
for i in range(500):
    dx = dyn(T)
    path = torch.cumsum(dx, dim=0) + x_0
    goal_loss = 1e2 * criterion(path[-1], x_f)
    dist_loss = torch.norm(dx, dim=1).nanmean()
    cost_loss = 1e-4 * cost(path).mean()
    loss = goal_loss + dist_loss + cost_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print(f'Loss: {loss.item()}')

In [ ]:
with torch.no_grad():
    dx = dyn(T)
path = torch.cumsum(dx, dim=0) + x_0

fig = go.Figure()
fig.add_trace(go.Scatter(x=path[:, 0].detach().cpu().numpy(), y=path[:, 1].detach().cpu().numpy(), mode='markers+lines'))
fig.update_layout(width=500, height=500, scene_aspectmode='data')
fig.show()